In [1]:
import fiftyone as fo
import fiftyone.zoo as foz

In [2]:
import os
from matplotlib import pyplot as plt
import cv2
import numpy as np

In [3]:
classes= ["Cat"]

In [4]:
dataset = foz.load_zoo_dataset(
    "open-images-v6",
    splits = ['train', 'validation'],
    label_types=["segmentations"],
    classes=classes,
    only_matching = True
)

Necessary images already downloaded
Existing download of split 'train' is sufficient
Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'open-images-v6-train-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [5]:
dataset

Name:        open-images-v6-train-validation
Media type:  image
Num samples: 10214
Persistent:  False
Tags:        []
Sample fields:
    id:             fiftyone.core.fields.ObjectIdField
    filepath:       fiftyone.core.fields.StringField
    tags:           fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    segmentations:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    open_images_id: fiftyone.core.fields.StringField

In [6]:
dataset.match_tags("validation", bool=True).tag_samples("val")

In [7]:
fo.launch_app(dataset)

Dataset:          open-images-v6-train-validation
Media type:       image
Num samples:      10214
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/

In [8]:
#Export detections in YOLO format
export_dir = "open_data"

for split in ["train", "val"]:
    split_view = dataset.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split
    )

 100% |███████████████| 9943/9943 [4.9m elapsed, 0s remaining, 32.8 samples/s]      
Directory 'open_data' already exists; export will be merged with existing files
 100% |█████████████████| 271/271 [8.3s elapsed, 0s remaining, 33.3 samples/s]       


In [11]:
# To ensure the order of the classes in the yolo configuration file
"""
yolo_dataset = fo.Dataset('yolo')
yolo_dataset.add_dir(
        dataset_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
)
classes = yolo_dataset.get_classes("all")
"""

# Define YOLO label map
label_map = {name: i for i, name in enumerate(classes)}

# Define function to convert segmentation labels to YOLO format
def to_yolo(seg_labels):
    # Create list to store YOLO format labels
    yolo_labels = []
    for label in seg_labels:
        mask = np.uint8(label.mask)        
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0].astype(np.float32)
        cnt = cnt.reshape(cnt.shape[0], 2)
        cnt[:,0] = cnt[:,0]/float(mask.shape[1])#sample.metadata.width)
        cnt[:,1] = cnt[:,1]/float(mask.shape[0])#(sample.metadata.height)
        cnt = cnt.reshape(cnt.shape[0]*2)
        
        label_boxes = label.bounding_box
        x, y, w, h = label_boxes
        class_id = label_map[label.label]
        yolo_labels.append((class_id, x, y, w, h, cnt))            
    return yolo_labels


# Iterate over samples in dataset and write YOLO format labels to file
for sample in dataset:
    # Get image path
    image_path = sample.filepath
    # Get segmentation labels
    seg_labels = sample.segmentations.detections
    # Compute metadata as width and height
    sample.compute_metadata()
    # Convert segmentation labels to YOLO format
    yolo_labels = to_yolo(seg_labels)
    # Define output file path
    filename = os.path.splitext(os.path.basename(image_path))[0] + ".txt"
    output_file = os.path.join(export_dir, 'labels', sample.tags[-1], filename)
    # Write YOLO format labels to file
    with open(output_file, "w") as f:
        for label in yolo_labels:
            class_id, x, y, w, h, mask = label
            # Write label to file in YOLO format
            f.write(f"{class_id} {x} {y} {w} {h} {' '.join(str(b) for b in mask)}\n")
            f.write(f"{class_id} {' '.join(str(b) for b in mask)}\n")
            #f.write(f"{class_id} {' '.join(str(b) for b in mask)}\n")